In [3]:
import sys
sys.path.append('/home/alexabades/recsys')


In [1]:
from time import time

start = time()

In [6]:
end = time()

In [9]:
(end-start)/60

1.8515099962552388

In [4]:
from src.models.mlp.mlp import MLP

EMBEDDING_SIZE = 3
N_USERS = 5
N_ITEMS = 10
a = MLP(2*EMBEDDING_SIZE, N_ITEMS, [6, 4, 2], dropout=0)

In [5]:
layers = [32, 23,45]
[int(layers[0] / 2)]

[16]

In [6]:
import numpy as np
from scipy.sparse import dok_matrix
S = dok_matrix((2, 2), dtype=np.float32)
for i in range(2):
    for j in range(2):
        
          S[i, j] = 1

In [7]:
print(S)

  (0, 0)	1.0
  (0, 1)	1.0
  (1, 0)	1.0
  (1, 1)	1.0


In [8]:
S.keys()

dict_keys([(0, 0), (0, 1), (1, 0), (1, 1)])

In [15]:
import pandas as pd
movies_path = '../data/ml-1m/movies.dat'



In [16]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [90]:
import pandas as pd

# Example paths, adjust as necessary
ratings_path = '../data/ml-1m/ratings.dat'
users_path = '../data/ml-1m/users.dat'
movies_path = '../data/ml-1m/movies.dat'

# Load the data
ratings = pd.read_csv(ratings_path, sep='::', engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
users = pd.read_csv(users_path, sep='::', engine='python', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
movies = pd.read_csv(movies_path, sep='::', engine='python', encoding='ISO-8859-1', names=['MovieID', 'Title', 'Genres'])


from sklearn.model_selection import train_test_split

train_dt, test = train_test_split(ratings, test_size=0.2, random_state=42)

from torch.utils.data import Dataset, DataLoader
import torch

class MovieLensDataset(Dataset):
    def __init__(self, ratings):
        self.users = torch.tensor(ratings['UserID'].values, dtype=torch.long)
        self.items = torch.tensor(ratings['MovieID'].values, dtype=torch.long)
        self.ratings = torch.tensor(ratings['Rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]


In [112]:
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

In [11]:
import json
def save_accuracy(filepath:str, **kwargs):
    
    with open('acuracy.json', 'w') as file:
        json.dump(kwargs, file)

In [12]:
filepath = "./src/checkpoints/nfc"

In [16]:
save_accuracy(filepath, a=5, b=3, c=4)

In [33]:
ratings2 = ratings[0:80]
ratings2.head(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760


In [134]:
ratings.iloc[-1]

UserID            6040
MovieID           1097
Rating               4
Timestamp    956715569
Name: 1000208, dtype: int64

In [51]:
ratings_path = "../data/processed/ml-1m.train.rating"
ratings = pd.read_csv(ratings_path, sep='\t', engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

ratings["MovieID"].max(), ratings["UserID"].max()

(3705, 6039)

In [56]:
import scipy.sparse as sp
num_users, num_items = 0, 0
with open(ratings_path, "r") as f:
    line = f.readline()
    
    while line != None and line != "":
        arr = line.split("\t")
        u, i = int(arr[0]), int(arr[1])
        num_users = max(num_users, u)
        num_items = max(num_items, i)
        line = f.readline()
# Construct matrix
mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
with open(ratings_path, "r") as f:
    line = f.readline()
    while line != None and line != "":
        arr = line.split("\t")
        user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
        if (rating > 0):
            mat[user, item] = 1.0
        line = f.readline()    

In [138]:
test["MovieID"]

895536    5412
899739    5440
55687      368
63727      425
822011    4942
          ... 
756007    4505
477775    2934
424188    2572
293600    1748
149989     965
Name: UserID, Length: 200042, dtype: int64

In [141]:
pairs_list = list(zip(test['UserID'], test['MovieID']))
torch.tensor(pairs_list, dtype=torch.long)

tensor([[5412, 2683],
        [5440,  904],
        [ 368, 3717],
        ...,
        [2572,  968],
        [1748, 1625],
        [ 965, 1233]])

In [1]:
import sys

sys.path.append('/home/alexabades/recsys')
from src.data.DataLoader import MovieLensDataset
data = MovieLensDataset()

In [2]:
path = "../data/processed/ml-1m/ml-1m"

In [12]:
import torch
a = [1,2,3]
b = []

In [13]:
b.extend(a)

In [14]:
b

[1, 2, 3]

In [4]:
data.load_processed_data(path)

In [8]:
import numpy as np
u = 4
items = [1]*20
users = np.full(len(items), u, dtype="int32")

In [9]:
users

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [75]:
train_dt = ratings[:400]
train_dt.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

In [63]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[1]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while train.has_key((u, j)):
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [65]:
import numpy as np
from scipy.sparse import csr_matrix
from typing import Tuple, List

def get_train_instances(train:sp.dok_matrix, num_negatives: int) -> Tuple[List[int], List[int], List[int]]:
    """
    Create nº Negative instances per postive instance in the matrix.

    Args:
        - train: Train sparse matrix storing the positive instances fro pairs (n_users, n_items).
        - num_negatives: The number of negative instances we want to generate per positive instacne.
    Reruns:
        - user_input: List of User IDs 
        - item_input: List of Items IDs
        - labels: Binary List determine Positive or Negative Instance
    """
    user_input, item_input, labels = [], [], []
    num_items = train.shape[1]
    # Iterate over all non-zero elements in the sparse matrix
    # which are the positive instances
    train_coo = train.tocoo()  # Convert to COO format for easy iteration
    for u, i in zip(train_coo.row, train_coo.col):
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # negative instances
        for _ in range(num_negatives):
            j = np.random.randint(num_items)
            # Keep generating a new negative item until we find one that 
            # the user has not interacted with
            while train[u, j] != 0:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    
    return user_input, item_input, labels


In [67]:
user_input, item_input, labels = get_train_instances(mat, 5)

In [71]:
def get_train_instances(train, num_items, num_negatives):
        user_input, item_input, labels = [], [], []
        num_users = train.shape[0]

        # Assuming 'train' is a dictionary with keys as (user, item) tuples
        for u, i in train.keys():
            # positive instance
            user_input.append(u)
            item_input.append(i)
            labels.append(1)

            # negative instances
            for _ in range(num_negatives):
                j = np.random.randint(num_items)
                while (u, j) in train:
                    j = np.random.randint(num_items)
                user_input.append(u)
                item_input.append(j)
                labels.append(0)

        # Converting lists to PyTorch tensors
        user_input = torch.tensor(user_input, dtype=torch.long)
        item_input = torch.tensor(item_input, dtype=torch.long)
        labels = torch.tensor(labels, dtype=torch.float)

        return user_input, item_input, labels


6039

In [126]:
mat

<6040x3706 sparse matrix of type '<class 'numpy.float32'>'
	with 994169 stored elements in Dictionary Of Keys format>

22384240

In [62]:
import torch

# Define the initial non-zero elements and their indices
initial_indices = torch.tensor([[0, 1, 2], [2, 0, 3]]) # Shape [2, N] where N is the number of non-zero elements
initial_values = torch.tensor([1, 2, 3])  # Non-zero values corresponding to the indices

# Define the size of the sparse tensor
size = torch.Size([3, 4])  # For example, a 3x4 matrix

# Create the initial sparse tensor
sparse_tensor = torch.sparse_coo_tensor(initial_indices, initial_values)
print("Initial Sparse Tensor:", sparse_tensor)

sparse_tensor[0]

Initial Sparse Tensor: tensor(indices=tensor([[0, 1, 2],
                       [2, 0, 3]]),
       values=tensor([1, 2, 3]),
       size=(3, 4), nnz=3, layout=torch.sparse_coo)


tensor(indices=tensor([[2]]),
       values=tensor([1]),
       size=(4,), nnz=1, layout=torch.sparse_coo)

In [50]:
num_users, num_items


(6039, 3705)

In [128]:
import pandas as pd
class MovieLensDataset(Dataset):
  def __init__(self, path:str, test_size:float=0.2, num_negatives:int=4) -> None:
    self.ratings = self._load_datasets(path)
    self.negative = "TODO"
    self.traindatset, self.testdataset = self._split_traintest(test_size)
    self.trainMartrix = self.trainMx(self.traindatset)
    self.test = "TODO"
    self.path = path 
    self.num_negatives = num_negatives

  def _load_datasets(self, ratings_path:str) ->pd.DataFrame:
    return pd.read_csv(ratings_path, sep='::', engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

  def _split_traintest(self, test_size: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Funtion that splits the dataser into train/test with a specific test size and a fixed seed for repr.

    Args:
      - test_size: float indicating the % of split of the data
    """
    train, test = train_test_split(self.ratings, test_size=test_size, random_state=42)
    return train, test
  
  def trainMx(self, train_dt: pd.DataFrame) -> sp.dok_matrix:    
    """
    Function that iterates through the dataset and generates a sparse matrix

    Args: 
      - train_dt: trainig data to prepare to sparse matrix
    Returns: 
      - sparse matrix storing the positive instances for pairs (n_users, n_items).
    """
    num_users = train_dt["UserID"].max()
    num_items = train_dt["MovieID"].max()
    # Construct matrix: Look out indexing
    mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    # Ensure that no 0 or lack of feedback is parsed as positve
    for _, row in train_dt.iterrows():
        user, item, rating = int(row['UserID']), int(row['MovieID']), float(row['Rating'])
        if rating > 0:
            mat[user, item] = 1.0
    return mat
  
  def get_train_instances(self, train:sp.dok_matrix, num_negatives: int) -> Tuple[List[int], List[int], List[int]]:
    """
    Create nº Negative instances per postive instance in the matrix.

    Args:
        - train: sparse matrix storing the positive instances for pairs (n_users, n_items).
        - num_negatives: The number of negative instances we want to generate per positive instacne.
    Reruns:
        - user_input: List of User IDs 
        - item_input: List of Items IDs
        - labels: Binary List determine Positive or Negative Instance
    """
    user_input, item_input, labels = [], [], []
    num_items = train.shape[1]
    # Iterate over all non-zero elements in the sparse matrix
    # which are the positive instances
    train_coo = train.tocoo()  # Convert to COO format for easy iteration
    for u, i in zip(train_coo.row, train_coo.col):
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # negative instances
        for _ in range(num_negatives):
            j = np.random.randint(num_items)
            # Keep generating a new negative item until we find one that 
            # the user has not interacted with
            while train[u, j] != 0:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    
   
    return user_input, item_input, labels
  
  def get_train(self):
    """
    Funtion that creates the necessary data to train the model. 

    Returns:
      - user_input: Tensor of dimensions (nº of UserdId x nº of ratigs per user x nº of negative samples + 1) | Also:  (length of train dataset * 5)
      - item_input: Tensor of dimensions (nº of UserdId x nº of ratigs per user x nº of negative samples + 1) | Also:  (length of train dataset * 5)
      - labels: Tensor of dimensions (nº of UserdId x nº of ratigs per user x nº of negative samples + 1) | Also:  (length of train dataset * 5)
    """
    
    user_input, item_input, labels = self.get_train_instances(self.trainMartrix, self.num_negatives)
     # Converting lists to PyTorch tensors
    user_input = torch.tensor(user_input, dtype=torch.long)
    item_input = torch.tensor(item_input, dtype=torch.long)
    labels = torch.tensor(labels, dtype=torch.float)
    
    return user_input, item_input, labels
  


In [104]:
ratings_path = '../data/ml-1m/ratings.dat'
data = MovieLensDataset(ratings_path)

In [106]:
user_input, item_input, labels = data.get_train()

In [118]:
len(user_input)

4000835

In [132]:
hidden_dims = [64,32,16,8]
layers = []
for layer in range(len(hidden_dims)-1):
            layers.append((hidden_dims[layer], hidden_dims[layer+1]))


In [133]:
layers

[(64, 32), (32, 16), (16, 8)]